# Step 3 - Prepare OD Estimates

In [39]:
import geopandas as gpd
import pandas as pd
import os, sys, time, importlib

import GOSTnets as gn
import importlib
# import Network_Clean as gnClean
importlib.reload(gn)

import networkx as nx
import osmnx
from shapely.geometry import Point
import numpy as np

import rasterio
from rasterio import features
from shapely.wkt import loads


#### Parameters

In [40]:
simplif_meters = 25

In [41]:
source_epsg = 4326
target_epsg = 3106

In [42]:
walk_speed = 4

Changing parameters

In [77]:
scenario = 'Current'
# scenario = 'Padma'

In [44]:
# WorldPop data determinants

constraint_status = 'constrained'
# constraint_status = 'unconstrained'

wp_res = 100
# wp_res = 250
# wp_res = '1k'

In [45]:
# Production date for outputs being used

prod_date = '210329'
# prod_date = '210503'

file paths

In [46]:
input_pth = r'..\..\..\GEO'

adm_pth = r'../../../GEO/Boundaries'
pop_pth = r'../../../GEO/Population/WorldPop'

fin_pth = 'final'
res_pth = 'results'

#### Define functions and variables

Cleaning functions

In [47]:
# function for sorting alphanumerically

import re

def sorted_nicely( l ): 
    """ Sort the given iterable in the way that humans expect.""" 
    convert = lambda text: int(text) if text.isdigit() else text 
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(l, key = alphanum_key)

# funciton for sorting matrices smallest to largest, by origin ID then destination ID

def sort_od_matrix(od_matrix):
    
    # sort by O_IDs, then dest node IDs
    od_matrix = od_matrix.sort_values('Unnamed: 0').reindex(sorted_nicely(od_matrix.columns), axis=1)

#     # reset O_ID column to the front
#     od_matrix = od_matrix[ ['Unnamed: 0'] + [ col for col in od_matrix.columns if col != 'Unnamed: 0' ] ]

    # set the Dest_ID column back to index so the shape is the same as the dWeight shape
    od_matrix.set_index('Unnamed: 0',inplace=True)

pickles

In [48]:
fin_pckle = f'final_{scenario}_G_{simplif_meters}m_{prod_date}.pickle'

In [50]:
G = nx.read_gpickle(os.path.join(fin_pth, fin_pckle))

origins

In [51]:
grid_name = f'bgd_wp_{constraint_status}_origins_{wp_res}m_2020_snapped_{simplif_meters}m.csv'
print(grid_name)
pop_origins = pd.read_csv(os.path.join(fin_pth,prod_date,grid_name))

bgd_wp_constrained_origins_100m_2020_snapped_25m.csv


In [52]:
len(pop_origins)

2199542

In [53]:
# Create dfs of destinations as possible origins, for dest-to-dest analysis

City_df = pd.read_csv(os.path.join(fin_pth,prod_date,\
                        f'All_cities_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification_snapped.csv'))

Dhaka_Chitt_df = pd.read_csv(os.path.join(fin_pth,prod_date,\
                        f'Dhaka_Chitt_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification_snapped.csv'))
Minor_cities_df = pd.read_csv(os.path.join(fin_pth,prod_date,\
                        f'Minor_cities_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification_snapped.csv'))
Dry_ports_df = pd.read_csv(os.path.join(fin_pth,prod_date,\
                        f'Dry_ports_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification_snapped.csv'))
River_ports_df = pd.read_csv(os.path.join(fin_pth,prod_date,\
                        f'River_ports_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification_snapped.csv'))
Deep_sea_ports_df = pd.read_csv(os.path.join(fin_pth,prod_date,\
                        f'Deep_sea_ports_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification_snapped.csv'))

All_SEZs_df = pd.read_csv(os.path.join(fin_pth,prod_date,\
                        f'All_SEZs_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification_snapped.csv'))

Active_SEZs_df = pd.read_csv(os.path.join(fin_pth,prod_date,\
                        f'Active_SEZs_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification_snapped.csv'))


destinations

In [54]:
# Destinations

city_fil = r'Population\\Places\\bd_major_cities_pop_projs.gpkg'
dhaka_chitt_fil = r'Population\\Places\\bd_cities_DhakaChittOnly_pop_projs.gpkg'
minor_city_fil = r'Population\\Places\\bd_cities_noDhakaChitt_pop_projs.gpkg'
dry_ports_fil = r'Economic\\Ports\\bd_landports_active.gpkg'
river_ports_fil = r'P:\BGD\GEO\Team\Infrastructure_Diagnostic\Nodes\river_ports.gpkg'
deep_sea_ports_fil = r'Economic\\Ports\\bd_DeepSeaPorts.gpkg'

sezs_all_fil = r'Economic\\SEZs\SEZs_all_pt_4326.gpkg'
sezs_active_fil = r'Economic\\SEZs\SEZs_active_pt_4326.gpkg'

In [82]:
# dests = {"All_cities" : city_fil, "Dhaka_Chitt" : dhaka_chitt_fil, "Minor_cities" : minor_city_fil, \
#          "Dry_ports" : dry_ports_fil, "River_ports" : river_ports_fil, "Deep_sea_ports" : deep_sea_ports_fil, \
#          'All_SEZs' : sezs_all_fil, 'Active_SEZs' : sezs_active_fil}

# dests = {"All_cities" : city_fil, "Dhaka_Chitt" : dhaka_chitt_fil, "Minor_cities" : minor_city_fil, \
#          "Dry_ports" : dry_ports_fil, "River_ports" : river_ports_fil, "Deep_sea_ports" : deep_sea_ports_fil}

# dests = {'All_SEZs' : sezs_all_fil, 'Active_SEZs' : sezs_active_fil}

dests = {"Dry_ports" : dry_ports_fil, "River_ports" : river_ports_fil, "Deep_sea_ports" : deep_sea_ports_fil}

scenarios (for looping)

In [83]:
origins = {'PopOrigins' : [G,f'_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification_snapped.csv',pop_origins,scenario],\
             'CityOrigins' : [G,f'_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification_snapped.csv',City_df,scenario]}

# origins = {'All_SEZ_Origins' : [G,f'_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification_snapped.csv',All_SEZs_df,f'{scenario}'], \
#              'Active_SEZ_Origins' : [G,f'_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification_snapped.csv',Active_SEZs_df,f'{scenario}']  }

#### Define functions that will generate OD matrices, and then generate direct walking times from origins to destinations

In [78]:
def od_routine(G_input,orig_file,orig_label,scen,snap_ending):
    
    # Start timer
    func_start = time.time()
    
    # main method
    
    origins_nodes = list(set(orig_file.NN)) # consolidates by shared nodes. 
    
    for dest_type, fpth in dests.items():

        snapfile = dest_type + snap_ending

        print(dest_type)

        dest = pd.read_csv(os.path.join(fin_pth, prod_date,snapfile))
        dest_nodes = list(set(dest.NN))

        print(len(list(set(dest.NN))))

        od_time = gn.calculate_OD(G_input, origins=origins_nodes, 
                              destinations=dest_nodes, fail_value=99999999, weight='time')

        od_time_df = pd.DataFrame(od_time, index=origins_nodes, columns=dest_nodes)

        print(od_time_df.shape)

        # Add walking time (from origin to NN) for each OD

        # origins_join = origins_pop_snapped.merge(od_time_df, how='left', on='NN')
        orig_file['NN_dist_seconds'] = ((orig_file.NN_dist / 1000) / walk_speed) * 60 * 60
        origins_join = orig_file.join(od_time_df, on='NN', rsuffix="dist_")

    #     print(origins_join.head())

        origins_join.columns[6:len(origins_join.columns)]

        # export
        
        origins_join.to_csv(os.path.join(fin_pth,prod_date,f'{orig_label}_to_{dest_type}_{scenario}_walktime_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification.csv'))
        od_time_df.to_csv(os.path.join(fin_pth,prod_date,f'OD_matrix_{scenario}_{orig_label}_to_{dest_type}_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification.csv'))
        
        # Report time
        
        func_end = time.time()
        print('time elapsed for function')
        print(str((func_end - func_start) / 60) + ' minutes')

In [79]:
def prepare_shortest_TT(snap_ending,origin_file,orig_label,scen='',calc_walk_time=False):
    
    # Start timer
    func_start = time.time()
    
    print(orig_label)

    # prep origin grid referencing origin file

    orig_grid = origin_file.rename(columns = {'Unnamed: 0' : 'Row_ID','NN':'O_ID','NN_dist':'walk_time_to_O_ID'})
    orig_grid = orig_grid.set_index(orig_grid['O_ID'])

    if calc_walk_time == True:
        orig_grid['geometry'] = orig_grid['geometry'].apply(loads)
    else:
        None

    # main method 
    for dest_type, fpth in dests.items():

        snapfile = dest_type + snap_ending

        print(dest_type)

        # Load OD matrix

        OD_name = f'OD_matrix_{scenario}_{orig_label}_to_{dest_type}_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification.csv'
        OD = pd.read_csv(os.path.join(fin_pth, prod_date, OD_name))
        
        # prepare OD matrix

        OD = OD.rename(columns = {'Unnamed: 0':'O_ID'})
        OD = OD.set_index('O_ID')
        OD = OD.replace([np.inf, -np.inf], np.nan)
        OD = OD.reindex(sorted_nicely(OD.columns), axis=1)

        # Load in snapfile for geoms, nodes, and dest-to-node walk distance

        dest_snap = pd.read_csv(os.path.join(fin_pth,prod_date,f'{dest_type}_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification_snapped.csv'))
        dest_snap.sort_values('NN',inplace=True)

        dest_nodes = dest_snap[['NN','NN_dist']]  # let's use the same file
        dest_nodes = dest_nodes.set_index('NN') # consider renaming something  more intuitive
        dest_nodes.index = dest_nodes.index.map(str)
        
        # transpose the OD matrix and join the data to each of the destination nodes

        odm_trans = OD.transpose()
        
        for i in odm_trans.columns:
            dest_nodes[i] = odm_trans[i] # assign ODM values to dest_nodes -- order correct?

#         for i in dest_nodes.columns:
#             if i == 'NN_dist':
#                 pass
#             else:
#                 dest_nodes[i] = dest_nodes[i] + dest_nodes['NN_dist'] # add walking time from dest point to nearest network node
                
        dest_nodes = dest_nodes.apply(lambda x: x + dest_nodes.NN_dist,axis=0)
        dest_nodes['NN_dist'] = dest_nodes['NN_dist'] / 2 # restore to original value. This is faster than for looping and if filtering.
                
        # Prepare dest_nodes, now with corresponding ODM data per dest, to be joined to the final_grid

        dest_nodes = dest_nodes.drop('NN_dist', axis = 1).transpose() # drop walk distance to nearest network node, now that it's included
        dest_nodes['on_network_time'] = dest_nodes.min(axis = 1) # select shortest travel time as min time
        dest_nodes['D_ID'] = dest_nodes.idxmin(axis=1) # calculation destination ID from column name
        dest_nodes.columns.names = ['']
        dest_nodes.index.names = ['O_ID']

        # Add walk time from population origin point to network origin point

        final_grid = pd.merge(orig_grid,dest_nodes[['D_ID','on_network_time']],how="left",left_index=True,right_index=True)

        final_grid['walk_time_to_O_ID'] = final_grid['walk_time_to_O_ID'] / 1000 * 3600 / walk_speed # refactor walking times to KMs and seconds
        final_grid['total_time_net'] = final_grid['on_network_time'] + final_grid['walk_time_to_O_ID']

        # Walking times from destination to network point -- optional subroutine
        
        if calc_walk_time == True:
            
            # prepare geodataframes of the origins (pop grid) and destinations (snap points) for walk time analysis
            o_2_d_walktime = gpd.GeoDataFrame(final_grid, crs = {'init':'epsg:{}'.format(source_epsg)}, geometry = 'geometry')

            dest_gdf = gpd.GeoDataFrame(dest_snap, crs = {'init':'epsg:4326'}, geometry = dest_snap['geometry'].apply(loads))
            dest_gdf = dest_gdf.rename({"NN" : "D_ID"},axis=1).set_index('D_ID').drop(['Unnamed: 0','Number'],axis=1)

            # Snapping to calculate direct walking times

            print('start of snapping: %s\n' % time.ctime())
            o_2_d_walktime = gn.pandana_snap_points(o_2_d_walktime, 
                                       dest_gdf,
                                       source_crs='epsg:{}'.format(source_epsg),
                                       target_crs='epsg:{}'.format(target_epsg),
                                       add_dist_to_node_col = True)
            print('\nend of snapping: %s' % time.ctime())
            print('\n--- processing complete')

            # Calculate walk time direct
            o_2_d_walktime['walk_time_direct'] = o_2_d_walktime['idx_dist'] / 1000 * 3600 / walk_speed
            final_grid['walk_time_direct'] = o_2_d_walktime['walk_time_direct']
            
            # Calculate final quickest time in seconds
            
            final_grid['PLOT_TIME_SECS'] = final_grid[['walk_time_direct','total_time_net']].min(axis = 1)
            
            # Specify which value was taken as the quickest
            
            def choice(x):
                if x.walk_time_direct < x.total_time_net:
                    return 'walk'
                else:
                    return 'net'

            final_grid['choice'] = final_grid.apply(lambda x: choice(x), axis = 1)
        
        # If skipping the subroutine, just calculate the seconds of travel from the total time
        
        else:
            final_grid['PLOT_TIME_SECS'] = final_grid[['total_time_net']]
            final_grid['choice'] = 'net' # to harmonize with datasets that differentiate between walking and net travel

        # Final calculations and cleanup
        
        final_grid['PLOT_TIME_MINS'] = final_grid['PLOT_TIME_SECS'] / 60

        final_grid = final_grid.drop('O_ID', axis = 1) # drop duplicate
        
        # Export

        final_grid.to_csv(os.path.join(fin_pth,prod_date,f'final_od_grid_{scenario}_{dest_type}_{orig_label}_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification.csv'))
        final_grid.to_csv(os.path.join(fin_pth,prod_date,f'final_od_grid_{scenario}_{dest_type}_{orig_label}_{constraint_status}_{wp_res}m_res_{simplif_meters}m_simplification.csv'))        
        
        # Report time
        
        func_end = time.time()
        print('time elapsed for function')
        print(str((func_end - func_start) / 60) + ' minutes')

### Load origins and destinations, get unique origin nodes, run OD, export

Run the OD routine function on all destinations for the 3 different scenarios

In [84]:
for orig, values in origins.items():
    
    print(len(values[2]))
    
    od_routine(values[0],\
               orig_file=values[2],\
               orig_label=orig,\
               scen=values[3],\
               snap_ending=values[1])

2199542
Dry_ports
16
(152099, 16)
time elapsed for function
6.219833568731944 minutes
River_ports
13
(152099, 13)
time elapsed for function
11.15425864458084 minutes
Deep_sea_ports
4
(152099, 4)
time elapsed for function
13.715617096424102 minutes
43
Dry_ports
16
(43, 16)
time elapsed for function
0.7144750038782756 minutes
River_ports
13
(43, 13)
time elapsed for function
1.3122584581375123 minutes
Deep_sea_ports
4
(43, 4)
time elapsed for function
1.4942334651947022 minutes


### Import completed OD matrix, calculate walking times from origins to the destinations

In [85]:
for orig, values in origins.items():
    
        prepare_shortest_TT(snap_ending=values[1],\
                            origin_file=values[2],\
                            orig_label=orig,\
                            scen=values[3],\
                            calc_walk_time=False)

PopOrigins
Dry_ports
time elapsed for function
36.72903771400452 minutes
River_ports
time elapsed for function
70.55681764682134 minutes
Deep_sea_ports
time elapsed for function
106.80096070369085 minutes
CityOrigins
Dry_ports
time elapsed for function
0.0049080491065979 minutes
River_ports
time elapsed for function
0.0077580650647481285 minutes
Deep_sea_ports
time elapsed for function
0.01041640043258667 minutes


## That's it